In [10]:
#!pip install --upgrade pip setuptools wheel
#!python -m pip install opencv-python
#!pip install matplotlib
#!pip freeze
#!pip install opencv-python-headless
#!pip install scikit-image
#!pip install -U scikit-fuzzy
#!pip install pandas
#!pip install scikit-learn
#!pip install tqdm==4.40.0

from matplotlib import pyplot as plt
import cv2
import os
import random
import pandas as pd
import numpy as np
from os.path import join
from skimage import measure
from multiprocessing import Process
from skimage.feature import blob_dog, blob_log, blob_doh
import skfuzzy as fuzz
import library

### GENERATING FEATURES

In [11]:
import library
import texture_features
import color_features
import glcm_features
import shape_features
import hair_removal
import pandas as pd
import numpy as np
import cv2
import os

def run_process(args):
    flag = True
    dictF = {}
    features = pd.DataFrame()
    count = 0
    
    idx = args[0]
    samples = args[1]
    for sample in samples:
        #print('count ', count)
        count += 1

        # read image
        img = cv2.imread(sample)

        # clahe preprocessing
        clahe = library.clahe_rgb(img, 8)
        
        # hair removal
        clahe = clahe.astype("uint8")
        hairless = hair_removal.hair_remove(img, 17, 4)
        
        dictF['name'] = sample
        dictF['label'] = (0 if 'nevus' in sample else 1 )

        # color features
        #colors = color_features.extract_color_features(hairless)
        #dictF.update(colors)

        # glcm features
        #angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
        #distances = [1]
        #colorspaces = ['rgb', 'hsv', 'lab', 'ycc', 'gray']

        #for cs in colorspaces:
        #    glcm = glcm_features.get_glcm(hairless, angles, distances, cs)
        #    dictF.update(glcm)

        # lbp features
        #lbp = texture_features.extract_lbp(hairless, 1, 8)
        #dictF.update(lbp)

        
        # orb features
        #orb = texture_features.extract_orb(hairless, 64)
        #dictF.update(orb)
        
        #zernike features
        hairless = cv2.cvtColor(hairless, cv2.COLOR_RGB2GRAY)
        zernike = shape_features.extract_zernike_moments(hairless, 200)
        dictF.update(zernike)
        
        features = features.append(dictF, ignore_index=True)

        # save features
        library.writeFeatures(features,
                      flag,
                      os.path.join('/home',
                         'emily',
                         'Desktop',
                         'CAD',
                         'MelanomaChallenge',
                         'features'),
                         'featuresCh2BTrainZ_{}.csv'.format(idx))


        flag = False
        
        del img
        del clahe
        del hairless
        del zernike
        del features
        
        features = pd.DataFrame()
        dictF.clear()
        
        import gc
        gc.collect()


In [12]:
flag = True

output = ""

allsamples, flag = library.get_sample_ch2("/home/emily/Desktop/CAD/challenge2/train",output, flag)

donesamples = pd.read_csv(os.path.join('/home',
                     'emily',
                     'Desktop',
                     'CAD',
                     'MelanomaChallenge',
                     'features',
                     'featuresCh2BTrainZ.csv'))['name'].values.tolist()

print("len donesamples ", len(donesamples))

missingsamples = library.uniques_names(allsamples, donesamples)

print("len missingsamples ", len(missingsamples))

all_processes = []
BATCH_SIZE = 6


def create_batches(iterable, n = 1):
    current_batch = []
    for item in iterable:
        current_batch.append(item)
        if len(current_batch) == n:
            yield current_batch
            current_batch = []
    if current_batch:
        yield current_batch

def prepare_process(samples):
    subsetsamples = np.array_split(samples, 40)#4800/8
    for idx, subsetsample in enumerate(subsetsamples):
        p = Process(target=run_process, args=([idx, subsetsample], ))
        all_processes.append(p)
    
    for batch in create_batches(all_processes,BATCH_SIZE):
        for process in batch:
            process.start()
        for process in batch:
            process.join()


prepare_process(missingsamples)

5082
len donesamples  1249
len missingsamples  3833


/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated 

/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated 

/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated 

/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated 

...

/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated 

/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated 

/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)
/tmp/ipykernel_12817/3171043836.py:64: FutureWarning: The frame.append method is deprecated 

KeyboardInterrupt: 

Process Process-106:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_12817/3171043836.py", line 32, in run_process
    hairless = hair_removal.hair_remove(img, 17, 4)
  File "/home/emily/Desktop/CAD/MelanomaChallenge/hair_removal.py", line 36, in hair_remove
    final_img = cv2.inpaint(img, hairless_img,7,cv2.INPAINT_TELEA)
KeyboardInterrupt
Process Process-102:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_12817/3171043836.py", line 32, in run_process
    hairless = hair_removal.hair_remove(img, 17, 4)
  File "/home/emily/Desktop/CAD/Melan

# MACHINE LEARNING

In [ ]:
import os
import library
import pandas as pd
from sklearn import tree
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif    

#
classifiers = ["rf", "tree", "svm", "adaboost", "gradboost", "histgradboost", "knn", "lda"]

train = pd.read_csv(os.path.join('/home','emily','Desktop','CAD','MelanomaChallenge','features','features_train_bh_3000.csv'))
test = pd.read_csv(os.path.join('/home','emily','Desktop','CAD','MelanomaChallenge','features','features_test_bh_3000.csv'))

y = train['label']
X = train.drop(['label'], axis=1)
X = X.drop(['name'], axis=1)


y_test = test['label']
X_test = test.drop(['label'], axis=1)
X_test = X_test.drop(['name'], axis=1)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=1)

# preprocessing options
#('selectFromModel', SelectFromModel(RandomForestClassifier(random_state=42, n_jobs = -1)))
#('selector rfe', RFE(RandomForestClassifier(random_state=42, n_jobs = -1))),
#('reduce_dims', PCA(n_components=150)),
#('mutual_info_classif, SelectKBest(mutual_info_classif, k=100)),

for classifier in classifiers:
    
    # preprocessing steps
    pipe = [('scale', StandardScaler()),
            ('selector rfe', RFE(RandomForestClassifier(random_state=42, n_jobs = -1)))
           ]

    
    if classifier == "svm":
        clf, best_params = library.SVC_linear(X_val, y_val, cv=2)
        clf.set_params(**best_params)
        print("### SVM ###")
    
    elif classifier == "rf":
        clf, best_params = library.RandomForest(X_val, y_val, cv=2)
        clf.set_params(**best_params)
        print("### RF ###")
    
    elif classifier == "tree":
        clf = tree.DecisionTreeClassifier()
        print("### TREE ###")
    
    elif classifier == "adaboost":
        clf, best_params = library.AdaBoost(X_val, y_val)
        clf.set_params(**best_params)
        print("### ADABOOST ###")
    
    elif classifier == "gradboost":
        clf, best_params = library.GradientBoosting(X_val, y_val)
        clf.set_params(**best_params)
        print("### GRADBOOST ###")
    
    elif classifier == "knn":
        clf, best_params = library.knn(X_val, y_val)
        clf.set_params(**best_params)
        print("### KNN ###")
        
    elif classifier == "histgradboost":
        clf = HistGradientBoostingClassifier()
        print("### HISTGRADBOOST ###")
        
    elif classifier == "lda":
        clf = LinearDiscriminantAnalysis()
        print("### LDA ###")        
        
    # add classifier 
    pipe.append(tuple(('clf', clf)))
    
    steps = Pipeline(pipe)
    
    # pipeline shape
    print("current pipeline")
    print(steps)
    
    library.fit_report(steps, X, y, X_test, y_test)
    